# 01 — Data Preparation & Market Construction

This notebook is part of the project **Quantitative Modeling of Financial Markets**, whose goal is to build a complete quantitative pipeline integrating econometrics, volatility modeling, risk measurement, derivative pricing, stochastic simulation, and portfolio optimization.

## Objectives of this notebook
- Collect and organize financial time series from multiple sources  
- Clean, align, and preprocess raw market data  
- Build a reliable price and return panel to be used throughout the rest of the project  

## Contents
1. Library imports  
2. Data loading and extraction (indices, FX, volatility, macro variables)  
3. Cleaning, alignment and preprocessing  
4. Preliminary diagnostics and data validation  
5. Partial conclusions  

In [7]:
from pathlib import Path

# Project Directory Configuration 

# Project root = one level above /notebooks/
PROJECT_ROOT = Path("..")

# Main data directories following ETL structure
DATA_DIR = PROJECT_ROOT / "data"
RAW_DIR = DATA_DIR / "raw"
PROCESSED_DIR = DATA_DIR / "processed"
POWERBI_DIR = DATA_DIR / "powerbi"

# Ensure folders exist (idempotent)
for folder in [RAW_DIR, PROCESSED_DIR, POWERBI_DIR]:
    folder.mkdir(parents=True, exist_ok=True)

print("Data directories ready:")
print("RAW_DIR      :", RAW_DIR)
print("PROCESSED_DIR:", PROCESSED_DIR)
print("POWERBI_DIR  :", POWERBI_DIR)


Data directories ready:
RAW_DIR      : ../data/raw
PROCESSED_DIR: ../data/processed
POWERBI_DIR  : ../data/powerbi


In [8]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

print("Using data folders:")
print("RAW_DIR      :", RAW_DIR)
print("PROCESSED_DIR:", PROCESSED_DIR)
print("POWERBI_DIR  :", POWERBI_DIR)


Using data folders:
RAW_DIR      : ../data/raw
PROCESSED_DIR: ../data/processed
POWERBI_DIR  : ../data/powerbi


In [9]:
# Asset universe definition and clean naming

# Mapping between Yahoo Finance tickers and standardized internal names
ticker_map = {
    "^IBEX": "IBEX35",
    "^GSPC": "SP500",
    "^NDX": "NASDAQ100",
    "^STOXX50E": "EUROSTOXX50",
    "EURUSD=X": "EURUSD",
    "^VIX": "VIX"
}

# Tickers to download from Yahoo Finance
tickers = list(ticker_map.keys())

# Time window for historical market data
start_date = "2000-01-01"
end_date = "2025-01-01"

print("Tickers loaded:", tickers)
print("Date range:", start_date, "to", end_date)

Tickers loaded: ['^IBEX', '^GSPC', '^NDX', '^STOXX50E', 'EURUSD=X', '^VIX']
Date range: 2000-01-01 to 2025-01-01


In [10]:
# Download raw adjusted prices from Yahoo Finance, (prices are already adjusted for splits and dividends)

# Download Close prices
prices_raw = yf.download(
    tickers,
    start=start_date,
    end=end_date,
    auto_adjust=True
)["Close"]

# Rename columns using project internal names
prices_raw = prices_raw.rename(columns=ticker_map)

# Clean and format index
prices_raw = prices_raw.sort_index()
prices_raw.index.name = "Date"

print(" Raw adjusted prices downloaded successfully:")
display(prices_raw.head())

# Export raw prices to ETL folder: data/raw/
raw_export_path = RAW_DIR / "prices_raw.csv"
prices_raw.to_csv(raw_export_path)

print(f" Raw prices exported to: {raw_export_path}")


[*********************100%***********************]  6 of 6 completed

✓ Raw adjusted prices downloaded successfully:


Ticker,EURUSD,SP500,IBEX35,NASDAQ100,EUROSTOXX50,VIX
Date,,,,,,
2000-01-03,NaN,1455.219971,11609.988281,3790.550049,NaN,24.209999
2000-01-04,NaN,1399.420044,11206.587891,3546.199951,NaN,27.010000
2000-01-05,NaN,1402.109985,10863.088867,3507.310059,NaN,26.410000
2000-01-06,NaN,1403.449951,NaN,3340.810059,NaN,25.730000
2000-01-07,NaN,1441.469971,11102.388672,3529.600098,NaN,21.719999


✓ Raw prices exported to: ../data/raw/prices_raw.csv


In [11]:
# Basic data cleaning and validation for price series

# Drop assets that are fully missing (ticker download failures)
prices_clean = prices_raw.dropna(axis=1, how="all")

# Forward-fill and backward-fill small missing segments
prices_clean = prices_clean.ffill().bfill()

# Sanity check: ensure no remaining gaps
print("Missing values after cleaning:")
display(prices_clean.isna().sum())

# Preview cleaned dataset
display(prices_clean.head())

# Export cleaned price series to data/processed/
clean_prices_path = PROCESSED_DIR / "clean_prices.csv"
prices_clean.to_csv(clean_prices_path)

print(f" Cleaned prices exported to: {clean_prices_path}")


Missing values after cleaning:


Ticker
EURUSD         0
SP500          0
IBEX35         0
NASDAQ100      0
EUROSTOXX50    0
VIX            0
dtype: int64

Ticker,EURUSD,SP500,IBEX35,NASDAQ100,EUROSTOXX50,VIX
Date,,,,,,
2000-01-03,1.196501,1455.219971,11609.988281,3790.550049,4181.029785,24.209999
2000-01-04,1.196501,1399.420044,11206.587891,3546.199951,4181.029785,27.010000
2000-01-05,1.196501,1402.109985,10863.088867,3507.310059,4181.029785,26.410000
2000-01-06,1.196501,1403.449951,10863.088867,3340.810059,4181.029785,25.730000
2000-01-07,1.196501,1441.469971,11102.388672,3529.600098,4181.029785,21.719999


✓ Cleaned prices exported to: ../data/processed/clean_prices.csv


In [19]:
# Export cleaned price data for downstream notebooks (N02+)
clean_prices_path = PROCESSED_DIR / "clean_prices.csv"
prices_clean.to_csv(clean_prices_path)

print(f" Cleaned prices exported to: {clean_prices_path}")


✓ Cleaned prices exported to: exports/processed/clean_prices.csv


In [12]:
# Compute simple returns and log returns

# Simple percentage returns
returns_simple = prices_clean.pct_change().dropna()

# Logarithmic returns (preferred for econometrics & risk modeling)
log_returns = np.log(prices_clean / prices_clean.shift(1)).dropna()

# Index formatting
returns_simple.index.name = "Date"
log_returns.index.name = "Date"

print("Simple returns preview:")
display(returns_simple.head())

print("\nLog returns preview:")
display(log_returns.head())

# Export returns to data/processed/
simple_returns_path = PROCESSED_DIR / "returns_simple.csv"
log_returns_path = PROCESSED_DIR / "log_returns.csv"

returns_simple.to_csv(simple_returns_path)
log_returns.to_csv(log_returns_path)

print(f" Simple returns exported to: {simple_returns_path}")
print(f" Log returns    exported to: {log_returns_path}")


Simple returns preview:


Ticker,EURUSD,SP500,IBEX35,NASDAQ100,EUROSTOXX50,VIX
Date,,,,,,
2000-01-04,0.0,-0.038345,-0.034746,-0.064463,0.0,0.115655
2000-01-05,0.0,0.001922,-0.030652,-0.010967,0.0,-0.022214
2000-01-06,0.0,0.000956,0.000000,-0.047472,0.0,-0.025748
2000-01-07,0.0,0.027090,0.022029,0.056510,0.0,-0.155849
2000-01-10,0.0,0.011190,0.006386,0.053210,0.0,-0.000460



Log returns preview:


Ticker,EURUSD,SP500,IBEX35,NASDAQ100,EUROSTOXX50,VIX
Date,,,,,,
2000-01-04,0.0,-0.039099,-0.035364,-0.066635,0.0,0.109441
2000-01-05,0.0,0.001920,-0.031131,-0.011027,0.0,-0.022464
2000-01-06,0.0,0.000955,0.000000,-0.048636,0.0,-0.026085
2000-01-07,0.0,0.026730,0.021790,0.054971,0.0,-0.169424
2000-01-10,0.0,0.011128,0.006366,0.051843,0.0,-0.000461


✓ Simple returns exported to: ../data/processed/returns_simple.csv
✓ Log returns    exported to: ../data/processed/log_returns.csv


In [13]:
# Create wide and long data formats
# Wide format -> one column per asset (modeling)
# Long format -> tidy structure for Power BI

# WIDE FORMAT (Python use)
prices_wide = prices_clean.reset_index()
returns_simple_wide = returns_simple.reset_index()
log_returns_wide = log_returns.reset_index()

# LONG FORMAT (Power BI)
prices_long = prices_clean.reset_index().melt(
    id_vars="Date",
    var_name="Asset",
    value_name="Price"
)

returns_simple_long = returns_simple.reset_index().melt(
    id_vars="Date",
    var_name="Asset",
    value_name="ReturnSimple"
)

log_returns_long = log_returns.reset_index().melt(
    id_vars="Date",
    var_name="Asset",
    value_name="ReturnLog"
)

# Previews
print("Prices (long format):")
display(prices_long.head())

print("\nSimple returns (long format):")
display(returns_simple_long.head())

print("\nLog returns (long format):")
display(log_returns_long.head())

# Export: Wide → data/processed/, Long → data/powerbi/

# WIDE EXPORTS (processed modeling data)
prices_wide.to_csv(PROCESSED_DIR / "prices_wide.csv", index=False)
returns_simple_wide.to_csv(PROCESSED_DIR / "returns_simple_wide.csv", index=False)
log_returns_wide.to_csv(PROCESSED_DIR / "log_returns_wide.csv", index=False)

# LONG EXPORTS (Power BI format)
prices_long.to_csv(POWERBI_DIR / "prices_long.csv", index=False)
returns_simple_long.to_csv(POWERBI_DIR / "returns_simple_long.csv", index=False)
log_returns_long.to_csv(POWERBI_DIR / "log_returns_long.csv", index=False)

print("\n Wide-format datasets exported to:", PROCESSED_DIR)
print(" Long-format datasets exported to:", POWERBI_DIR)


Prices (long format):


,Date,Asset,Price
0,2000-01-03,EURUSD,1.196501
1,2000-01-04,EURUSD,1.196501
2,2000-01-05,EURUSD,1.196501
3,2000-01-06,EURUSD,1.196501
4,2000-01-07,EURUSD,1.196501



Simple returns (long format):


,Date,Asset,ReturnSimple
0,2000-01-04,EURUSD,0.0
1,2000-01-05,EURUSD,0.0
2,2000-01-06,EURUSD,0.0
3,2000-01-07,EURUSD,0.0
4,2000-01-10,EURUSD,0.0



Log returns (long format):


,Date,Asset,ReturnLog
0,2000-01-04,EURUSD,0.0
1,2000-01-05,EURUSD,0.0
2,2000-01-06,EURUSD,0.0
3,2000-01-07,EURUSD,0.0
4,2000-01-10,EURUSD,0.0



✓ Wide-format datasets exported to: ../data/processed
✓ Long-format datasets exported to: ../data/powerbi


# Summary of Notebook N01 — Data Preparation & Market Construction

In this notebook, we built the foundational dataset that will be used throughout the quantitative pipeline. 

The goal was to construct a clean a panel of financial time series, suitable for econometric modeling, volatility analysis, risk estimation, derivative pricing and portfolio optimization.

## Key steps completed

### 1. Environment setup and directory structure
We defined relative paths for the project and created dedicated folders for:
- raw data  
- processed data  
- Power BI exports  

### 2. Selection of the asset universe
We defined a multi-asset dataset including major equity indices, an FX rate and a volatility index:
- IBEX 35  
- S&P 500  
- NASDAQ 100  
- EURO STOXX 50  
- EUR/USD  
- VIX  

### 3. Download of adjusted market prices
Using `yfinance` (with `auto_adjust=True`), we retrieved adjusted prices that incorporate dividends and corporate actions.  
We then standardized column names and ensured a clean chronological ordering.

### 4. Data cleaning and validation
We removed empty columns, filled small gaps with forward/backward filling, and verified that no missing values remained after preprocessing.

### 5. Construction of returns
We computed:
- simple percentage returns, useful for descriptive analysis  
- log returns, recommended for econometrics and time-series modeling  

Both representations were indexed clearly and validated.

### 6. Creation of wide and long data formats
To support both Python modeling and Power BI visualization, we generated:
- **wide format** tables (one column per asset)  
- **long tidy format** tables (`Date–Asset–Value`)  

### 7. Export of all datasets
All datasets were exported as CSV files into the appropriate project directories:
- `/data/raw/`  
- `/data/processed/`  
- `/data/powerbi/`  

These exports will serve as inputs for later notebooks and for Power BI dashboards.

## Asset Selection and Rationale

The dataset used in this project includes a set of financial assets chosen to reflect the core dynamics of global markets. Each asset contributes a dimension that is necessary for econometric modelling, volatility analysis, risk measurement, derivative pricing and portfolio construction.

### **Assets included**
- **IBEX 35**  
  Represents the Europe equity market.

- **S&P 500 (SPX)**  
  One of the most representative indices of global equity performance. 

- **NASDAQ 100**  
  A technology-heavy index with higher volatility and momentum characteristics. 

- **EURO STOXX 50**  
  Captures the economic dynamics of the Eurozone and is often cointegrated with IBEX 35.

- **EUR/USD exchange rate**  
  Adds a macro-financial component to the dataset.

- **VIX (CBOE Volatility Index)**  
  A proxy for market fear and expected volatility.

### **Why these assets**

1. **Diversity of market behaviour**  
   The combination of US, European, FX and volatility indices generates rich dynamics for modeling returns, correlations and market regimes.

2. **Econometric relationships**  
   Assets such as IBEX 35 and EURO STOXX 50 are often cointegrated, while SP500 and NASDAQ 100 show both long-term comovement and volatility clustering.

3. **High liquidity and data reliability**  
   All selected assets have deep markets and long historical price series, ensuring stability and robustness in statistical analysis.
